In [ ]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import time
import sys

#Set device
USE_CUDA = torch.cuda.is_available()
if USE_CUDA:
    device = torch.device("cuda")
    cuda = True
else:
    device = torch.device("cpu")
    cuda = False
    
print("Device =", device)
gpus = [0]

In [ ]:
def time_elapsed(start_time):
    elapsed = time.time() - start_time
    hours = int(elapsed/3600)
    minutes = int(int(elapsed/60)%60)
    seconds = int(elapsed%60)
    
    return hours, minutes, seconds

In [ ]:
#Load SMILES data as integer labels and as one-hot encoding
data = np.load("ohefinemols.npz")
data = data["arr_0"]

intdata = np.load("intfinemols.npz")
intdata = intdata["arr_0"]

data = torch.from_numpy(data).view(np.shape(data)[0], 1, np.shape(data)[1])
intdata = torch.from_numpy(intdata)

print("Dataset size: " + str(data.size()))
print("Integer dataset size: " + str(intdata.size()))

In [ ]:
#Get input tensor
def inptarg(i, batch_size):
    while True:
        try:
            #Define input and target tensor sizes
            inp = torch.Tensor((seq_length - 1) * batch_size, 1, np.shape(data)[2])
            target = torch.Tensor((seq_length - 1) * batch_size)
    
            #SMILES molecules in batch 
            inputs = data[int((seq_length * i)) : int((seq_length * i) + (seq_length * batch_size)), :, :]
            targets = intdata[int((seq_length * i)) : int((seq_length * i) + (seq_length * batch_size))]

            #Index counters for input, target
            r = 0
            s = 0 
            for p in range(seq_length * batch_size - 1):
        
                if (p % seq_length != (seq_length - 1)):
                    #Input data (does not include last character in SMILES)
                    inp[r, :, :] = inputs[p, :, :]
                    r += 1
            
                if (p % seq_length != 0):
                    #Target data (does not include first character in SMILES)
                    target[s] = targets[p]
                    s += 1
                    
            return inp, target
        
        except:
            continue
        break

In [ ]:
#Define model
class Model(nn.Module):
    
    #Define model parameters
    def __init__(self, input_size, hidden_size, num_layers, dropout):
        super(Model, self).__init__()
        
        #Model parameters
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout = dropout
        
        #Model layers
        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, dropout = dropout)
        self.linear = nn.Linear(hidden_size, input_size)
        
    #Define initial hidden and cell states
    def init_states(self, num_layers, hidden_size):
        hidden = [Variable(torch.zeros(num_layers, 1, hidden_size)),
                  Variable(torch.zeros(num_layers, 1, hidden_size))]
         
        #Initialize forget gate bias to 1    
        for names in self.lstm._all_weights:
            for name in filter(lambda n: "bias" in n, names):
                bias = getattr(self.lstm, name)
                n = bias.size(0)
                start, end = n//4, n//2
                
                nn.init.constant_(bias.data[start:end], 1.0)
        
        return hidden
    
    #Define forward propagation
    def forward(self, inp, hidden):
        #LSTM
        output, hidden = self.lstm(inp, hidden)

        #Linear Layer
        output = self.linear(output)
        
        return output, hidden

In [ ]:
#Define training
def train():
    #Set start time
    start_time = time.time()
        
    #Iterate set of seq_length characters
    for i in range(int((np.shape(data)[0] - seq_length + 1) / batch_size)):
            
        #Initialize hidden and cell states
        hidden = model.init_states(num_layers, hidden_size)
            
        #Run on GPU if available
        if cuda:
            hidden = (hidden[0].cuda(), hidden[1].cuda())
        
        #Set initial gradients
        model.zero_grad()
    
        #Set initial loss
        loss = 0 
            
        #Get input and target
        input_data, target_data = inptarg(i, batch_size)
        input_data = input_data.float()
        target_data = target_data.long()
            
        #Run on GPU if available
        if cuda:
            input_data = input_data.cuda()
            target_data = target_data.cuda()
                
        #Run model, calculate loss
        output, hidden = model(input_data, hidden)
        loss += criterion(output.squeeze(), target_data.squeeze())
                
        #Backpropagate loss
        loss.backward()
        
        #Clip gradients
        nn.utils.clip_grad_norm_(model.parameters(), 3.0)
        
        #Optimize
        optimizer.step()
            
        #Update list of losses
        if (i % 50 == 0):
            losses[0] = loss.data.item() / seq_length   
        losses.append(loss.data.item() / seq_length)
            
        #Intermediary saves
        if (i % 250 == 0):
            torch.save(model.state_dict(), "network1.pth")

        #Print training info
        hours, minutes, seconds = time_elapsed(start_time)
        print("Loss: {:0.6f}".format(loss.data.item() / seq_length) + " | ΔLossTotal: {:+0.4f}".format(losses[-1] - losses[1]) + " | Iteration: {0:04d}".format(i + 1) + " | Time elapsed: {0:02d}".format(hours) + "h {0:02d}".format(minutes) + " m {0:02d}".format(seconds) + " s")

In [ ]:
#Initialize model parameters
input_size = np.shape(data)[2]
hidden_size = 1024
num_layers = 3
dropout = .2
learning_rate = 0.001
seq_length = 75
batch_size = 128

#List of losses
losses = [0]

In [ ]:
#Call model, set optimizer and loss function
model = Model(input_size, hidden_size, num_layers, dropout)
model.load_state_dict(torch.load("network.pth"))
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
criterion = nn.CrossEntropyLoss()

#Run on GPU if available
if cuda:
    model.cuda()
    criterion.cuda()

In [ ]:
#Total number of parameters
total_params = sum(p.numel() for p in model.parameters())
print("Total number of parameters in network: " + str(total_params))

In [ ]:
#Train
train()

In [ ]:
torch.save(model.state_dict(), "network1.pth")